In [1]:
import datetime
import os
import sys
import math
import pandas as pd
import numpy as np
from pprint import pprint
import json
import requests
import datetime
from datetime import date
import os
from pandas.io.json import json_normalize

In [2]:
# input for file naming convention
initial_value = "Anne_v2"

In [3]:
# raw data file naming conventions
# Anne file:   oura_2020-08-19_AG
# Janice file:  oura_2020-08-24_Janice
# Mike: MK_ouraraw_2020-07-22
# Beth: BF_ouradata

#v2: shift root node rotate_x = 90, branch level 1 rod translate_x = 270

In [4]:
with open('oura_2020-08-19_AG.json') as json_data:
    dataJ = json.load(json_data)
    
sleep = json_normalize(dataJ, 'sleep')
dfs = pd.DataFrame(sleep)

readiness = json_normalize(dataJ, 'readiness')
dfr = pd.DataFrame(readiness)

activity = json_normalize(dataJ, 'activity')
dfa = pd.DataFrame(activity)

FileNotFoundError: [Errno 2] No such file or directory: 'oura_2020-08-19_AG.json'

In [ ]:
df = dfa.merge(dfr, on='summary_date')

In [ ]:
df = df.merge(dfs, on = 'summary_date')
df.rename( columns={'Unnamed: 0':'index'}, inplace=True )

## Adjust selection of most dates if less than 25 in data set

In [ ]:
df = df.iloc[-42:]
df.index = np.arange(len(df))
df.shape
#df.columns.values.tolist()

# Section finding earliest sleep time & number of 5 min increments

In [ ]:
df['bedtime_start'] = pd.to_datetime(df['bedtime_start'])
# ask user's timezone
df['bedtime_start'] = df['bedtime_start'].dt.tz_localize('UTC').dt.tz_convert('US/Pacific')
print(df['bedtime_start'].head(2))

In [ ]:
gg = len(df.index)
hh = np.arange(0, gg, 1)

In [ ]:
# set up new dataframe with bedtime_start to get only those values prior to midnight
    # to find earliest sleep
ii = df['bedtime_start'].tolist()
dft = pd.DataFrame({'A': hh, 'B': ii})
dft = dft.set_index(pd.DatetimeIndex(ii))
dft.index = dft.index.tz_localize(None)

In [ ]:
dft = dft.between_time(start_time = '16:00:00', end_time = '23:59:59')
#print(dft)

In [ ]:
dft['BB'] = [d.time() for d in dft['B']]

In [ ]:
#earlist bedtime of this person's oura data in this export
mm = str(dft['BB'].min())
print(mm)

In [ ]:
#recreate dft to include all times
ii = df['bedtime_start'].tolist()
dft = pd.DataFrame({'A': hh, 'B': ii})
dft = dft.set_index(pd.DatetimeIndex(ii))
dft.index = dft.index.tz_localize(None)
dft['BB'] = [d.time() for d in dft['B']]

In [ ]:
# column 'BBtsmin' is the timestamp of the unique earliest bedtime for this json export
timestamp_bedtime_start = (pd.to_datetime('2020-01-01' + ' ' + str(mm)))
dft['BBtsmin'] = timestamp_bedtime_start
dft['BBtsmin'] = (dft['BBtsmin'].astype(int) / 10**9)

In [ ]:
llist = []
for index, row in dft.iterrows():
    if (row['BB'].hour > 12):
        # random day + 1
        x = '2020-01-01' + ' ' + str(row['BB'])
        llist.append(x)
    elif (row['BB'].hour < 12):
        # random day
        #rr = pd.to_datetime['2020-01-01' + ' ' + str(row['BB'])]
        x = x = '2020-01-02' + ' ' + str(row['BB'])
        llist.append(x)


In [ ]:
# column 'BB' is the time value of each day's bedtime start, ie 9:14pm
# column 'BBfull' is the datetime value using random day plus bedtime start
dft['BBfull'] = pd.to_datetime(llist)

In [ ]:
# timestamp of the random date + real bedtime start to get difference between earliest start
dft['BBts'] = (dft['BBfull'].astype(int) / 10**9) 
#print(dft[['BBts', 'BBtsmin', 'BBfull']])

In [ ]:
# BBdelta is the column with # of 5 min increments that need to shift to the right 
# to align rows with earliest sleep time
dft['BBdelta'] = (dft['BBts']/300) - (dft['BBtsmin']/300)
dft['BBdeltaint'] = dft['BBdelta'].round(0).astype('Int64')
print(dft['BBdeltaint'].head(1))

In [ ]:
# column 'daystart' is the timestamp of the unique earliest day start for all json exports
timestamp_day_start = (pd.to_datetime('2020-01-01' + ' ' + '04:00:00'))
dft['daystart'] = timestamp_day_start
dft['daystart'] = (dft['daystart'].astype(int) / 10**9)

In [ ]:
# column 'aspace' is the number of 5 min increments that need to be added to left of column 0
dft['aspace'] = (dft['BBtsmin']/300) - (dft['daystart']/300)
dft['aspace'] = dft['aspace'].round(0).astype('Int64')
print(dft['aspace'].head(1))

In [ ]:
dft = dft.reset_index()
dft = dft.drop(['index', 'A', 'B', 'BB', 'BBtsmin', 'BBfull', 'BBts', 'BBdelta', 'daystart'], axis = 1)
print(dft.head(1))

In [ ]:
BBdeltaint = dft['BBdeltaint']

In [ ]:
aivalue = dft['aspace'][0]
aivalue = int(aivalue)

In [ ]:
print(aivalue)

## Weekday code added to original df

In [ ]:
df['day_start'] = pd.to_datetime(df['day_start'], utc=True)
#print(df.day_start.dtype)  # datetime64[ns, UTC]
df['weekday'] = df['day_start'].dt.dayofweek
#print(df['weekday'])
weekday = df['weekday']

##  Converting [activity] met_5min parameter

In [ ]:
f = lambda x: 'met_{}'.format(x+1)

In [ ]:
dfmet = pd.DataFrame(df.met_1min.values.tolist(), df.index, dtype=object).fillna('').rename(columns=f)

In [ ]:
#dfmet.head()
#dfmet.isnull().sum().sum()
length = len(dfmet.index)
print(length)

## Converting [sleep] hr_5min parameter

In [ ]:
l = lambda x: 'hr_{}'.format(x+1)

In [ ]:
dfhr = pd.DataFrame(df.hr_5min.values.tolist(), df.index, dtype=object).fillna(0).rename(columns=l)

In [ ]:
dfhr.head(2)
dfhr.shape
#dfhr.isnull().sum().sum()

In [ ]:
dfhr = dfhr.join(BBdeltaint)

In [ ]:
for index, row in dfhr.iterrows():
    dfhr.loc[index,:] = dfhr.loc[index,:].shift(periods = row['BBdeltaint']).fillna(0)

In [ ]:
newcolshr = np.arange(0, aivalue, 1)
nclhr = [str('hradjust_') + str(item) for item in newcolshr]

In [ ]:
dfhr = dfhr.reindex(columns =  nclhr + dfhr.columns.tolist()).fillna(0)

In [ ]:
hrlen = len(dfhr.columns)

In [ ]:
nclist2 = np.arange(0, hrlen, 1)
ncl2 = [str('hr_') + str(item) for item in nclist2]
dfhr = dfhr.reindex(columns = ncl2).fillna(0)

## Converting [sleep] rmssd_5min parameter (hrv)

In [ ]:
m = lambda x: 'hrv_{}'.format(x+1)

In [ ]:
dfhrv = pd.DataFrame(df.rmssd_5min.values.tolist(), df.index, dtype=object).fillna(0).rename(columns=m)

In [ ]:
dfhrv.head(5)
dfhrv.astype('int64').dtypes
dfhrv.dtypes.value_counts()

In [ ]:
dfhrv = dfhrv.join(BBdeltaint)

In [ ]:
for index, row in dfhrv.iterrows():
    dfhrv.loc[index,:] = dfhrv.loc[index,:].shift(periods = row['BBdeltaint']).fillna(0)

In [ ]:
newcolshrv = np.arange(0, aivalue, 1)
nclhrv = [str('hrvadjust_') + str(item) for item in newcolshrv]

In [ ]:
dfhrv = dfhrv.reindex(columns =  nclhrv + dfhrv.columns.tolist()).fillna(0)

In [ ]:
hrvlen = len(dfhrv.columns)

In [ ]:
nclist3 = np.arange(0, hrvlen, 1)
ncl3 = [str('hrv_') + str(item) for item in nclist3]
dfhrv = dfhrv.reindex(columns = ncl3).fillna(0)

## Converting [sleep] hypnogram_5min

In [ ]:
#start
def split(value):
    return [char for char in value] 

In [ ]:
# fix this, right now generates a bunch of useless lists prior to the one we need
for column in df[['hypnogram_5min']]:
    col_list = df[column].tolist()
    hgfm=[]
    for i in col_list:
        i_s = str(i)
        value=i_s
        x=(split(value))
        hgfm.append(x)
        #print(hgfm)

In [ ]:
q = len(df.index)

In [ ]:
hgfm=hgfm[:q]

In [ ]:
aa=lambda x: 'hg_{}'.format(x+1)

In [ ]:
dfhgfm = pd.DataFrame(hgfm, df.index, dtype=object).fillna('5').rename(columns=aa)

In [ ]:
dfhgfm = dfhgfm.join(BBdeltaint)

In [ ]:
for index, row in dfhgfm.iterrows():
    dfhgfm.loc[index,:] = dfhgfm.loc[index,:].shift(periods = row['BBdeltaint']).fillna(5)

In [ ]:
newcolshgfm = np.arange(0, aivalue, 1)
nclhgfm = [str('hgfmadjust_') + str(item) for item in newcolshgfm]

In [ ]:
dfhgfm = dfhgfm.reindex(columns =  nclhgfm + dfhgfm.columns.tolist()).fillna(5)

In [ ]:
hgfmlen = len(dfhgfm.columns)

In [ ]:
nclist4 = np.arange(0, hgfmlen, 1)
ncl4 = [str('hg_') + str(item) for item in nclist4]
dfhgfm = dfhgfm.reindex(columns = ncl4).fillna(5)

In [ ]:
dfhgfm = dfhgfm.join(weekday)

## Converting [activity] class_5min parameter

In [ ]:
for column in df[['class_5min']]:
    col_list = df[column].tolist()
    c5m=[]
    for i in col_list:
        i_s = str(i)
        value=i_s
        x=(split(value))
        c5m.append(x)
        #print(c5m)

In [ ]:
r = len(df.index)

In [ ]:
#from list of lists, extract last list with all rows of original df
c5m=c5m[:r]

In [ ]:
h=lambda x: 'c_{}'.format(x+1)

In [ ]:
dfc5m = pd.DataFrame(c5m, df.index, dtype=object).fillna('5').rename(columns=h)
dfc5m = dfc5m.join(weekday)
dfc5m.shape

# Converting parameters for scaling conversion

In [ ]:
def make_interpolater(left_min, left_max, right_min, right_max): 
    # Figure out how 'wide' each range is  
    leftSpan = left_max - left_min  
    rightSpan = right_max - right_min  

    # Compute the scale factor between left and right values 
    scaleFactor = float(rightSpan) / float(leftSpan) 

    # create interpolation function using pre-calculated scaleFactor
    def interp_fn(value):
        return right_min + (value-left_min)*scaleFactor


    return interp_fn

## Scaling x for met_1min

In [ ]:
dfmetscalex = pd.DataFrame()

In [ ]:
for column in dfmet:
    columnSeriesObj = dfmet[column]
    col_min = dfmet[column].min() 
    col_max = dfmet[column].max() 
    col_list = dfmet[column].tolist()
    scalar = make_interpolater(col_min +.0002, col_max, .9, 5.8) 
    col_interp = [scalar(x) for x in col_list] 
    dfmetscalex[(str(column) + str('_scalex'))] = col_interp

In [ ]:
dfmetscalex = dfmetscalex.join(weekday)
#print(dfmetscalex.head())

## Scaling xyz for rmssd_5min

In [ ]:
dfhrvxyz = pd.DataFrame()

In [ ]:
for column in dfhrv:
    columnSeriesObj = dfhrv[column]
    col_min = dfhrv[column].min() 
    col_max = dfhrv[column].max() 
    col_list = dfhrv[column].tolist()
    scalar = make_interpolater(col_min +.0002, col_max, .1, 2.6) 
    col_interp = [scalar(x) for x in col_list] 
    dfhrvxyz[(str(column) + str('_scalexyz'))] = col_interp

In [ ]:
dfhrvxyz = dfhrvxyz.join(weekday)

## Scaling x for hr_5min

In [ ]:
dfhrxyz = pd.DataFrame()

In [ ]:
for column in dfhr:
    columnSeriesObj = dfhr[column]
    col_min = dfhr[column].min() 
    col_max = dfhr[column].max() 
    col_list = dfhr[column].tolist()
    scalar = make_interpolater(col_min +.0002, col_max, 1.5, 4.8) 
    col_interp = [scalar(x) for x in col_list] 
    dfhrxyz[(str(column) + str('_scalex'))] = col_interp

## Scaling y for hr_5min

In [ ]:
for column in dfhr:
    columnSeriesObj = dfhr[column]
    col_min = dfhr[column].min() 
    col_max = dfhr[column].max() 
    col_list = dfhr[column].tolist()
    scalar = make_interpolater(col_min +.0002, col_max, .35, 1.5) 
    col_interp = [scalar(x) for x in col_list] 
    dfhrxyz[(str(column) + str('_scaley'))] = col_interp

## Scaling z for hr_5min

In [ ]:
for column in dfhr:
    columnSeriesObj = dfhr[column]
    col_min = dfhr[column].min() 
    col_max = dfhr[column].max() 
    col_list = dfhr[column].tolist()
    scalar = make_interpolater(col_min +.0002, col_max, .18, .75) 
    col_interp = [scalar(x) for x in col_list] 
    dfhrxyz[(str(column) + str('_scalez'))] = col_interp

In [ ]:
dfhrxyz = dfhrxyz.join(weekday)

## Set up lvl2toroids number for each parameter

In [ ]:
lvl2met = [col for col in dfmet if col.startswith('met')]
lvl2met = len(lvl2met)
print(lvl2met)

In [ ]:
lvl2class = [col for col in dfc5m if col.startswith('c')]
lvl2class = len(lvl2class)
print(lvl2class)

In [ ]:
lvl2hg = [col for col in dfhgfm if col.startswith('hg')]
lvl2hg = len(lvl2hg)
print(lvl2hg)

In [ ]:
lvl2hrv = [col for col in dfhrv if col.startswith('hrv')]
lvl2hrv = len(lvl2hrv)
print(lvl2hrv)

In [ ]:
lvl2hr = [col for col in dfhr if col.startswith('hr')]
lvl2hr = len(lvl2hr)
print(lvl2hr)

# Set up axis data 

## met_1min axis

In [ ]:
# dfmet axes 
# spacing back, per day is the 3rd value in x axis arange.
# multiply that chosen spacing criteria by the # of days of data in the dataframe, lenviz

xa = list(range(-210, 210, 60))
x = xa*6

y1 = [41.5]*7
y2 = [22.5]*7
y3 = [3.5] *7
y4 = [-17.5]*7
y5 = [-36.5]*7
y6 = [-55.5]*7
y = y1 + y2 + y3 + y4 + y5 + y6

z = 3
dfmetscalex['x'] = x
dfmetscalex['y'] = y
dfmetscalex['z'] = z

## class_5min axis

In [ ]:
# dfc5m axes
xa = list(range(-210, 210, 60))
x = xa*6

y1 = [45]*7
y2 = [26]*7
y3 = [7] *7
y4 = [-14]*7
y5 = [-33]*7
y6 = [-52]*7
y = y1 + y2 + y3 + y4 + y5 + y6

z = 3
dfc5m['x'] = x
dfc5m['y'] = y
dfc5m['z'] = z
#print(x)
#print(z)

## hypnogram_5min axis

In [ ]:
# dfhgfm axes
xa = list(range(-210, 210, 60))
x = xa*6

y1 = [48.5]*7
y2 = [29.5]*7
y3 = [10.5] *7
y4 = [-10.5]*7
y5 = [-29.5]*7
y6 = [-48.5]*7
y = y1 + y2 + y3 + y4 + y5 + y6

z = 3
dfhgfm['x'] = x
dfhgfm['y'] = y
dfhgfm['z'] = z

## rmssd_5min axis

In [ ]:
# dfhrvxyz axes
xa = list(range(-210, 210, 60))
x = xa*6

y1 = [52]*7
y2 = [33]*7
y3 = [14] *7
y4 = [-7]*7
y5 = [-26]*7
y6 = [-45]*7
y = y1 + y2 + y3 + y4 + y5 + y6

z = 3
dfhrvxyz['x'] = x
dfhrvxyz['y'] = y
dfhrvxyz['z'] = z
#print(x)
#print(z)

## hr_5min axis

In [ ]:
# dfhrxyz axes
xa = list(range(-210, 210, 60))
x = xa*6

y1 = [55.5]*7
y2 = [36.5]*7
y3 = [17.5] *7
y4 = [-3.5]*7
y5 = [-22.5]*7
y6 = [-41.5]*7
y = y1 + y2 + y3 + y4 + y5 + y6

z = 3
dfhrxyz['x'] = x
dfhrxyz['y'] = y
dfhrxyz['z'] = z
#print(x)
#print(z)

## Set baseline node file values

In [ ]:
# set DEBUG flag
DEBUG = 0

# Initialize all ANTz Node variables
# You shouldn't need to change anything in this section
# you can find an explanation of these terms in Appendix B of the user manual
id = 0
type = 5
data = id
selected = 0
parent_id = 0
branch_level = 0
child_id = id
child_index = 0
palette_id = 0
ch_input_id = 0
ch_output_id = 0
ch_last_updated = 0
average = 0
samples = 0
aux_a_x = 0
aux_a_y = 0
aux_a_z = 0
aux_b_x = 0
aux_b_y = 0
aux_b_z = 0
color_shift = 0
rotate_vec_x = 0
rotate_vec_y = 0
rotate_vec_z = 0
rotate_vec_s = 1
scale_x = 0
scale_y = 0
scale_z = 0
translate_x = 0
translate_y = 0
translate_z = 0
tag_offset_x = 0
tag_offset_y = 0
tag_offset_z = 0
rotate_rate_x = 0
rotate_rate_y = 0
rotate_rate_z = 0
rotate_x = 0
rotate_y = 0
rotate_z = 0
scale_rate_x = 0
scale_rate_y = 0
scale_rate_z = 0
translate_rate_x = 0
translate_rate_y = 0
translate_rate_z = 0
translate_vec_x = 0
translate_vec_y = 0
translate_vec_z = 0
shader = 0
geometry = 7
line_width = 1
point_size = 0
ratio = 0.1
color_index = 0
color_r = 110
color_g = 110
color_b = 110
color_a = 255
color_fade = 0
texture_id = 0
hide = 0
freeze = 0
topo = 3
facet = 0
auto_zoom_x = 0
auto_zoom_y = 0
auto_zoom_z = 0
trigger_hi_x = 0
trigger_hi_y = 0
trigger_hi_z = 0
trigger_lo_x = 0
trigger_lo_y = 0
trigger_lo_z = 1
set_hi_x = 0
set_hi_y = 0
set_hi_z = 0
set_lo_x = 0
set_lo_y = 0
set_lo_z = 0
proximity_x = 0
proximity_y = 0
proximity_z = 0
proximity_mode_x = 0
proximity_mode_y = 0
proximity_mode_z = 0
segments_x = 16
segments_y = 16
segments_z = 0
tag_mode = 0
format_id = 0
table_id = 0
record_id = id
size = 420

In [ ]:
#met_1min node files
mn = str(initial_value) + "ouradaily_met_node.csv"

#met_1min tag files
mt = str(initial_value) + "ouradaily_met_tag.csv"

# class node files
cn = str(initial_value) + "ouradaily_class_node.csv"

# class tag files
ct = str(initial_value) + "ouradaily_class_tag.csv"

# hypnogram node files
hgn = str(initial_value) + "ouradaily_hypnogram_node.csv"

# hypnogram tag files
hgt = str(initial_value) + "ouradaily_hypnogram_tag.csv"

# hrv node files
hrvn = str(initial_value) + "ouradaily_hrv_node.csv"

# hrv tag files
hrvt = str(initial_value) + "ouradaily_hrv_tag.csv"

# hr node files
hrn = str(initial_value) + "ouradaily_hr_node.csv"

# hr tag files
hrt = str(initial_value) + "ouradaily_hr_tag.csv"

## Node file for met_1min

In [ ]:
# Open ANTz Node file for writing
fout = open(mn,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(mt,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,6,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6

numlevel0toroids = 1
numlevel1toroids = 1

for index, row in dfmetscalex.iterrows() :
    id += 1
    data = id
    type = 5
    parent_id = 0
    selected = 0
    branch_level = 0
    child_id = id
    translate_x = row['x']
    translate_y = row['y']
    translate_z = row['z']
    scale_x = .5 
    scale_y = scale_x
    scale_z = scale_x
    rotate_x = 90
    ratio = .1
    line_width= 2
    geometry = 3
    topo = 2
    if row['weekday'] == 1:
        color_r = 255
        color_g = 250
        color_b = 240
    if row['weekday'] == 2:
        color_r = 205
        color_g = 92
        color_b = 92
    if row['weekday'] == 3:
        color_r = 50
        color_g = 205
        color_b = 50
    if row['weekday'] == 4:
        color_r = 255
        color_g = 255
        color_b = 224
    if row['weekday'] == 5:
        color_r = 240
        color_g = 248
        color_b = 255
    if row['weekday'] == 6:
        color_r = 102
        color_g = 51
        color_b = 153
    if row['weekday'] == 7:
        color_r = 128
        color_g = 0
        color_b = 0
    color_a = 155
    rotate_x = 0
    rotate_y = 0
    rotate_z = 0
    record_id = id 
    texture_id= 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = "MET levels every 1 minute/ Day: " + str(row["weekday"])
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1

    level0objectid = id
    
    for j in range(numlevel1toroids):
        id += 1
        data = id
        type = 5
        parent_id = level0objectid
        selected = 0
        branch_level = 1
        child_id = id
        translate_x = 270
        translate_y = 0
        translate_z = 0
        scale_x = 8.54
        scale_y = 12
        scale_z = 42.74
        line_width= 1
        geometry = 19
        topo = 6
        color_r = 191
        color_g = 191
        color_b = 191
        color_a = 200
        rotate_x = 0
        rotate_y = 0
        rotate_z = 90
        ratio = .03
        record_id = id 
        texture_id= 0

        # Creat output string and write to ANTz Node File
        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
        fout.write(outputstring)

        level1objectid = id
        numlevel2toroids = lvl2met
        #cylinder objects for each reading per day
        for k in range(numlevel2toroids):
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            column = str((str("met_") + str(k+1) + str("_scalex")))
            scale_x = row[column] 
            scale_y = .14
            scale_z = .14 
            translate_x = -180 + (.12*k)
            translate_y = 0
            translate_z = 0
            rotate_x = 90
            rotate_y = 90
            rotate_z = 0
            ratio = .1
            line_width= 1
            geometry = 19
            topo = 4
            color_r = 106
            color_g = 189
            color_b = 139
            color_a = 255
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)


## Node file for class_5min

In [ ]:
# class_5min
# Open ANTz Node file for writing
fout = open(cn,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(ct,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,6,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6

numlevel0toroids = 1
numlevel1toroids = 1

for index, row in dfc5m.iterrows() :
    id += 1
    data = id
    type = 5
    parent_id = 0
    selected = 0
    branch_level = 0
    child_id = id
    translate_x = row['x']
    translate_y = row['y']
    translate_z = row['z']
    scale_x = .5 
    scale_y = scale_x
    scale_z = scale_x
    rotate_x = 90
    ratio = .1
    line_width= 2
    geometry = 3
    topo = 2
    if row['weekday'] == 1:
        color_r = 255
        color_g = 250
        color_b = 240
    if row['weekday'] == 2:
        color_r = 205
        color_g = 92
        color_b = 92
    if row['weekday'] == 3:
        color_r = 50
        color_g = 205
        color_b = 50
    if row['weekday'] == 4:
        color_r = 255
        color_g = 255
        color_b = 224
    if row['weekday'] == 5:
        color_r = 240
        color_g = 248
        color_b = 255
    if row['weekday'] == 6:
        color_r = 102
        color_g = 51
        color_b = 153
    if row['weekday'] == 7:
        color_r = 128
        color_g = 0
        color_b = 0
    color_a = 155
    rotate_x = 0
    rotate_y = 0
    rotate_z = 0
    record_id = id 
    texture_id= 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = "Class of activity every 5 minutes/ Day: " + str(row["weekday"])
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1

    level0objectid = id
    
    for j in range(numlevel1toroids):
        id += 1
        data = id
        type = 5
        parent_id = level0objectid
        selected = 0
        branch_level = 1
        child_id = id
        translate_x = 270
        translate_y = 0
        translate_z = 0
        scale_x = 8.54
        scale_y = 12
        scale_z = 42.74
        line_width= 1
        geometry = 19
        topo = 6
        color_r = 191
        color_g = 191
        color_b = 191
        color_a = 200
        rotate_x = 0
        rotate_y = 0
        rotate_z = 90
        ratio = .03
        record_id = id 
        texture_id= 0

        # Creat output string and write to ANTz Node File
        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
        fout.write(outputstring)

        level1objectid = id
        numlevel2toroids = lvl2class
        #cylinder objects for each reading per day
        for k in range(numlevel2toroids):
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            scale_x = 3.53
            scale_y = .93
            scale_z = .71 
            translate_x = -180 + (.6*k)
            translate_y = 0
            translate_z = 0
            rotate_x = 0
            rotate_y = 0
            rotate_z = 0
            ratio = .1
            line_width= 1
            geometry = 3
            topo = 2
            column = str(str("c_") + str(k+1))
            if row[column] == "1":
                color_r = 178
                color_g = 191
                color_b = 80
            elif row[column] == "2":
                color_r = 93
                color_g = 115
                color_b = 45
            elif row[column] == "3":
                color_r = 48
                color_g = 64
                color_b = 25
            elif row[column] == "4":
                color_r = 217
                color_g = 108
                color_b = 108
            elif row[column] == "5":
                color_r = 242
                color_g = 215
                color_b = 208
            color_a = 255
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)


## Node file for hypnogram_5min

In [9]:
# hypnogram_5min
# Open ANTz Node file for writing
fout = open(hgn,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(hgt,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,6,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6

numlevel0toroids = 1
numlevel1toroids = 1

for index, row in dfhgfm.iterrows() :
    id += 1
    data = id
    type = 5
    parent_id = 0
    selected = 0
    branch_level = 0
    child_id = id
    translate_x = row['x']
    translate_y = row['y']
    translate_z = row['z']
    scale_x = .5 
    scale_y = scale_x
    scale_z = scale_x
    rotate_x = 90
    ratio = .1
    line_width= 2
    geometry = 3
    topo = 2
    if row['weekday'] == 1:
        color_r = 255
        color_g = 250
        color_b = 240
    if row['weekday'] == 2:
        color_r = 205
        color_g = 92
        color_b = 92
    if row['weekday'] == 3:
        color_r = 50
        color_g = 205
        color_b = 50
    if row['weekday'] == 4:
        color_r = 255
        color_g = 255
        color_b = 224
    if row['weekday'] == 5:
        color_r = 240
        color_g = 248
        color_b = 255
    if row['weekday'] == 6:
        color_r = 102
        color_g = 51
        color_b = 153
    if row['weekday'] == 7:
        color_r = 128
        color_g = 0
        color_b = 0
    color_a = 155
    rotate_x = 0
    rotate_y = 0
    rotate_z = 0
    record_id = id 
    texture_id= 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = "Sleep stage/ Weekday: " + str(row["weekday"])
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1

    level0objectid = id
    
    for j in range(numlevel1toroids):
        id += 1
        data = id
        type = 5
        parent_id = level0objectid
        selected = 0
        branch_level = 1
        child_id = id
        translate_x = 270
        translate_y = 0
        translate_z = 0
        scale_x = 8.54
        scale_y = 12
        scale_z = 42.74
        line_width= 1
        geometry = 19
        topo = 6
        color_r = 191
        color_g = 191
        color_b = 191
        color_a = 200
        rotate_x = 0
        rotate_y = 0
        rotate_z = 90
        ratio = .03
        record_id = id 
        texture_id= 0

        # Creat output string and write to ANTz Node File
        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
        fout.write(outputstring)

        level1objectid = id
        numlevel2toroids = lvl2hg
        #spheres for each 5 min of sleep
        for k in range(numlevel2toroids):
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            scale_x =  1.48 
            scale_y = 1.48
            scale_z = .65
            translate_x = -180 + (.6*k)
            translate_y = 0
            translate_z = 0
            rotate_x = 0
            rotate_y = 0
            rotate_z = 0
            ratio = .1
            line_width= 1
            geometry = 3
            topo = 2
            column = str(str("hg_") + str(k))
            if row[column] == "1":
                color_r = 60
                color_g = 116
                color_b = 166
                color_a = 255
            elif row[column] == "2":
                color_r = 102
                color_g = 184
                color_b = 217
                color_a = 255
            elif row[column] == "3":
                color_r = 217
                color_g = 84
                color_b = 54
                color_a = 255
            elif row[column] == "4":
                color_r = 188
                color_g = 191
                color_b = 96
                color_a = 255
            elif row[column] == "5":
                color_r = 217
                color_g = 162
                color_b = 95
                color_a = 105
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)


NameError: name 'hgn' is not defined

## Node file for rmssd_5min

In [ ]:
# rmssd_5min (hrv)
# Open ANTz Node file for writing
fout = open(hrvn,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(hrvt,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,6,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6

numlevel0toroids = 1
numlevel1toroids = 1

for index, row in dfhrvxyz.iterrows() :
    id += 1
    data = id
    type = 5
    parent_id = 0
    selected = 0
    branch_level = 0
    child_id = id
    translate_x = row['x']
    translate_y = row['y']
    translate_z = row['z']
    scale_x = .5 
    scale_y = scale_x
    scale_z = scale_x
    rotate_x = 90
    ratio = .1
    line_width= 2
    geometry = 3
    topo = 2
    if row['weekday'] == 1:
        color_r = 255
        color_g = 250
        color_b = 240
    elif row['weekday'] == 2:
        color_r = 205
        color_g = 92
        color_b = 92
    elif row['weekday'] == 3:
        color_r = 50
        color_g = 205
        color_b = 50
    elif row['weekday'] == 4:
        color_r = 255
        color_g = 255
        color_b = 224
    elif row['weekday'] == 5:
        color_r = 240
        color_g = 248
        color_b = 255
    elif row['weekday'] == 6:
        color_r = 102
        color_g = 51
        color_b = 153
    elif row['weekday'] == 7:
        color_r = 128
        color_g = 0
        color_b = 0
    color_a = 155
    rotate_x = 0
    rotate_y = 0
    rotate_z = 0
    record_id = id 
    texture_id= 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = "HRV every 5 min/ Day: " + str(row["weekday"])
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1

    level0objectid = id
    
    for j in range(numlevel1toroids):
        id += 1
        data = id
        type = 5
        parent_id = level0objectid
        selected = 0
        branch_level = 1
        child_id = id
        translate_x = 270
        translate_y = 0
        translate_z = 0
        scale_x = 8.54
        scale_y = 12
        scale_z = 42.74
        line_width= 1
        geometry = 19
        topo = 6
        color_r = 191
        color_g = 191
        color_b = 191
        color_a = 200
        rotate_x = 0
        rotate_y = 0
        rotate_z = 90
        ratio = .03
        record_id = id 
        texture_id= 0

        # Creat output string and write to ANTz Node File
        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
        fout.write(outputstring)

        level1objectid = id
        numlevel2toroids = lvl2hrv
        #pin objects for each reading per day
        for k in range(numlevel2toroids):
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            column = str(str("hrv_") + str(k) + str("_scalexyz"))
            scale_x = row[column] 
            scale_y = scale_x
            scale_z = scale_x
            translate_x = -180 + (.6*k)
            translate_y = 0
            translate_z = 0
            rotate_x = 90
            rotate_y = 270
            rotate_z = 0
            ratio = .1
            line_width= 1
            geometry = 16
            topo = 5
            color_r = 242
            color_g = 29
            color_b = 181
            color_a = 255
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)


## Node file for hr_5min

In [ ]:
#hr_5min 
# Open ANTz Node file for writing
fout = open(hrn,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(hrt,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,6,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6

numlevel0toroids = 1
numlevel1toroids = 1

for index, row in dfhrxyz.iterrows() :
    id += 1
    data = id
    type = 5
    parent_id = 0
    selected = 0
    branch_level = 0
    child_id = id
    translate_x = row['x']
    translate_y = row['y']
    translate_z = row['z']
    scale_x = .5 
    scale_y = scale_x
    scale_z = scale_x
    rotate_x = 90
    ratio = .1
    line_width= 2
    geometry = 3
    topo = 2
    if row['weekday'] == 1:
        color_r = 255
        color_g = 250
        color_b = 240
    if row['weekday'] == 2:
        color_r = 205
        color_g = 92
        color_b = 92
    if row['weekday'] == 3:
        color_r = 50
        color_g = 205
        color_b = 50
    if row['weekday'] == 4:
        color_r = 255
        color_g = 255
        color_b = 224
    if row['weekday'] == 5:
        color_r = 240
        color_g = 248
        color_b = 255
    if row['weekday'] == 6:
        color_r = 102
        color_g = 51
        color_b = 153
    if row['weekday'] == 7:
        color_r = 128
        color_g = 0
        color_b = 0
    color_a = 155
    rotate_x = 0
    rotate_y = 0
    rotate_z = 0
    record_id = id 
    texture_id= 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = "HR every 5 min / Day: " + str(row["weekday"])
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1

    level0objectid = id
    
    for j in range(numlevel1toroids):
        id += 1
        data = id
        type = 5
        parent_id = level0objectid
        selected = 0
        branch_level = 1
        child_id = id
        translate_x = 270
        translate_y = 0
        translate_z = 0
        scale_x = 8.54
        scale_y = 12
        scale_z = 42.74
        line_width= 1
        geometry = 19
        topo = 6
        color_r = 191
        color_g = 191
        color_b = 191
        color_a = 200
        rotate_x = 0
        rotate_y = 0
        rotate_z = 90
        ratio = .03
        record_id = id 
        texture_id= 0

        # Creat output string and write to ANTz Node File
        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
        fout.write(outputstring)

        level1objectid = id
        numlevel2toroids = lvl2hr
        #cone objects for each reading per day
        for k in range(numlevel2toroids):
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            column = str((str("hr_") + str(k) + str("_scalex")))
            scale_x = row[column] 
            column = str((str("hr_") + str(k) + str("_scaley")))
            scale_y = row[column]
            column = str((str("hr_") + str(k) + str("_scalez")))
            scale_z = row[column] 
            translate_x = -180 + (.6*k)
            translate_y = 0
            translate_z = 0
            rotate_x = 90
            rotate_y = 90
            rotate_z = 0
            ratio = .1
            line_width= 1
            geometry = 5
            topo = 4
            color_r = 242
            color_g = 58
            color_b = 41
            color_a = 255
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)
